In [ ]:
%load_ext sql

In [ ]:
%%sql

CREATE OR REPLACE VIEW daily_product_revenue_v
AS
SELECT pd.product_id,
    pd.product_name,
    pd.product_category_id,
    pd.category_name,
    pd.category_department_id,
    pd.department_name,
    dpr.order_date,
    round(sum(dpr.revenue), 2) AS revenue
FROM daily_product_revenue AS dpr
    JOIN product_dim AS pd
        ON dpr.order_item_product_id = pd.product_id
WHERE dpr.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY pd.product_id,
    pd.product_name,
    pd.product_category_id,
    pd.category_name,
    pd.category_department_id,
    pd.department_name,
    dpr.order_date
ORDER BY dpr.order_date,
    revenue DESC;

In [ ]:
%%sql

SELECT * FROM daily_product_revenue_v LIMIT 10;

In [ ]:
%%sql

SELECT count(*) FROM daily_product_revenue_v;

In [ ]:
%%sql

SELECT order_date,
    product_id,
    product_name,
    revenue,
    dense_rank() OVER (PARTITION BY order_date ORDER BY revenue DESC) AS drnk
FROM daily_product_revenue_v
ORDER BY order_date, drnk
LIMIT 10;

In [ ]:
%%sql

WITH 
    daily_ranked_products AS (
        SELECT order_date,
            product_id,
            product_name,
            revenue,
            dense_rank() OVER (PARTITION BY order_date ORDER BY revenue DESC) AS drnk
        FROM daily_product_revenue_v
    )
SELECT * FROM daily_ranked_products
WHERE drnk <= 3
ORDER BY order_date,
    drnk
LIMIT 10;

In [ ]:
%%sql

SELECT * FROM (
    SELECT order_date,
        product_id,
        product_name,
        revenue,
        dense_rank() OVER (PARTITION BY order_date ORDER BY revenue DESC) AS drnk
    FROM daily_product_revenue_v
)
WHERE drnk <= 3
ORDER BY order_date,
    drnk
LIMIT 10;